In [28]:
# from ultralytics import YOLO
# import numpy as np
# import cv2
# import os
# import matplotlib.pyplot as plt

# # Define class names based on your model
# class_names = ["Striped_Hyena", "Fishing_cat"]  # Add other class names if necessary

# # Load the ONNX model
# onnx_model = YOLO("./best.onnx")

# # Function to crop the image based on bounding box coordinates
# def crop_image(image_path, bbox_coords, class_name, output_folder="C:/Users/Aarzo/Projects/Wildlife-Species-Identification/cropped_images"):
#     # Read the image
#     img = cv2.imread(image_path)
    
#     # Crop each bounding box region and save as a new image
#     for i, bbox in enumerate(bbox_coords):
#         x_min, y_min, x_max, y_max = map(int, bbox)
#         cropped_img = img[y_min:y_max, x_min:x_max]
        
#         # Ensure class-specific output folder exists
#         class_folder = os.path.join(output_folder, class_name)
#         os.makedirs(class_folder, exist_ok=True)
        
#         # Construct the output path
#         base_name = os.path.basename(image_path)
#         output_path = os.path.join(class_folder, f"{i}_{base_name}")  # Add index to avoid name collision
        
#         # Save the cropped image
#         cv2.imwrite(output_path, cropped_img)
#         print(f"Cropped image saved to: {output_path}")

# # Directory containing .jpg images
# input_folder = "path_to_Fishing-cat"
# output_folder = "path_to_cropped_images"

# # Iterate over each .jpg file in the input folder
# for file_name in os.listdir(input_folder):
#     if file_name.lower().endswith('.jpg'):
#         image_path = os.path.join(input_folder, file_name)
        
#         # Perform prediction on the image
#         results = onnx_model(image_path)
        
#         # Collect bounding boxes and class names
#         for result in results:
#             print(f"Processing Image: {image_path}")
            
#             # Collect bounding boxes by class
#             detections_by_class = {name: [] for name in class_names}
            
#             # Iterate over detections
#             for i, box in enumerate(result.boxes):
#                 cls = int(box.cls.item())  # Class index (convert to int for indexing)
#                 conf = box.conf.item()  # Confidence score
#                 xyxy = box.xyxy.cpu().numpy().tolist()[0]  # Bounding box coordinates
                
#                 if conf > 0.5:  # Threshold for confidence
#                     # Retrieve class name using the index
#                     class_name = class_names[cls] if cls < len(class_names) else "Unknown"
                    
#                     print(f"Detection {i + 1}:")
#                     print(f"  Class: {class_name}")
#                     print(f"  Confidence: {conf:.2f}")  # Format confidence to 2 decimal places
#                     print(f"  Bounding Box: {xyxy}")
#                     print("----------------------------------------------")
                    
#                     # Append bounding box coordinates by class
#                     detections_by_class[class_name].append(xyxy)
            
#             # Crop and save the image for each class detected
#             for class_name, bbox_coords in detections_by_class.items():
#                 if bbox_coords:  # Only crop if there are detections for the class
#                     crop_image(image_path, bbox_coords, class_name, output_folder)


In [ ]:
from ultralytics import YOLO
import numpy as np
import cv2
import os

# Define class names based on your model
class_names = ["Striped_Hyena", "Fishing_cat"]  # Add other class names as necessary

# Load the ONNX model
onnx_model = YOLO("./best.onnx")

# Function to crop and save the detected animals
def crop_and_save(image_path, bbox_coords, class_name, output_folder):
    # Read the image
    img = cv2.imread(image_path)
    
    # Crop each bounding box from the image
    for bbox in bbox_coords:
        x_min, y_min, x_max, y_max = map(int, bbox)
        cropped_img = img[y_min:y_max, x_min:x_max]
        
        # Ensure the output subfolder exists
        subfolder = os.path.basename(os.path.dirname(image_path))
        class_output_folder = os.path.join(output_folder, subfolder)
        os.makedirs(class_output_folder, exist_ok=True)
        
        # Construct the output path
        base_name = os.path.basename(image_path)
        output_path = os.path.join(class_output_folder, base_name)
        
        # Save the cropped image
        cv2.imwrite(output_path, cropped_img)
        print(f"Cropped image saved to: {output_path}")

# Input and output folders
input_folder = "path_to_Fishing_cat_folder"
output_folder = "path_to_Fishing-cat-cropped_folder"

# Iterate over subfolders and process each image
for subdir, _, files in os.walk(input_folder):
    for file_name in files:
        if file_name.lower().endswith('.jpg'):
            image_path = os.path.join(subdir, file_name)
            
            # Perform prediction on the image
            results = onnx_model(image_path)
            
            # Collect bounding boxes and class names
            for result in results:
                print(f"Processing Image: {image_path}")
                
                # Collect bounding boxes by class
                detections_by_class = {name: [] for name in class_names}
                
                # Iterate over detections
                for i, box in enumerate(result.boxes):
                    cls = int(box.cls.item())  # Class index (convert to int for indexing)
                    conf = box.conf.item()  # Confidence score
                    xyxy = box.xyxy.cpu().numpy().tolist()[0]  # Bounding box coordinates
                    
                    if conf > 0.5 and cls == 1:  # 1 corresponds to 'Fishing_cat'
                        print(f"  Detection {i + 1}:")
                        print(f"  Class: Fishing_cat")
                        print(f"  Confidence: {conf:.2f}")
                        print(f"  Bounding Box: {xyxy}")
                        print("----------------------------------------------")
                        
                        # Append bounding box coordinates for 'Fishing_cat'
                        detections_by_class["Fishing_cat"].append(xyxy)
                
                # Crop and save only if there are 'Fishing_cat' detections
                if detections_by_class["Fishing_cat"]:
                    crop_and_save(image_path, detections_by_class["Fishing_cat"], "Fishing_cat", output_folder)
